# Node.js in Jupyter Notebooks

![diagram](pixiedust.png)

* Using the [pixiedust_node](https://github.com/ibm-watson-data-lab/pixiedust_node) Jupyter add-on
* Inspiration: 
    - [Part I (Intro)](https://medium.com/ibm-watson-data-lab/nodebooks-node-js-data-science-notebooks-aa140bea21ba)
    - [Part II (Sharing data between NodeJS & Python)](https://medium.com/ibm-watson-data-lab/nodebooks-sharing-data-between-node-js-python-3a4acae27a02)
    - [Part III (Visualization)](https://medium.com/ibm-watson-data-lab/nodebooks-visualising-data-the-node-js-way-d6cb7c9d6540)

In [1]:
%%capture
!pip install pixiedust;

In [2]:
%%capture
!pip install pixiedust_node;

In [3]:
import pixiedust_node

Pixiedust database opened successfully


pixiedust_node 0.2.5 started. Cells starting '%%node' may contain Node.js code.


# Introduction - Printing JS variables

In [4]:
%%node
console.log('test')

test


In [5]:
%%node
const x = 'xxx'
const y = 'yyy'
console.log(x,y)

xxx yyy


In [6]:
%%node
var date = new Date();
print(date);

"2019-03-25T20:12:39.877Z"


In [7]:
%%node
var z = { a:1, b:2, c:3 };
print(z);

{"a": 1, "b": 2, "c": 3}


# Using PixieDust display() to visualize data

In [8]:
%%node
var data = [];
for (var i =0; i<1000; i++) {
    var n = 2*Math.PI*i/360;
    var obj = {
        n:n, i:i,
        sin: Math.sin(n), cos: Math.cos(n), tan: Math.tan(n) 
    };
    data.push(obj);
}
display(data);

# Adding NPM modules
* npm call is from a Python cell.
* once install, you can require the module in your JS code.
* let's try making HTTP calls to an external API using the npm __request__ module.

In [9]:
npm.install('request')

/home/bjpcjp/.nvm/versions/node/v10.6.0/bin/npm install -s request
+ request@2.88.0
updated 1 package and audited 287 packages in 4.227s
found 0 vulnerabilities


* HTTP request = an async action; the request library uses a __callback__ once done. Use __print__ to render inside the callback.
* in this case request the current position of the ISS (International Space Station).

In [11]:
%%node
var request = require('request');
var r = {
    method:'GET',
    url: 'http://api.open-notify.org/iss-now.json',
    json: true
};
request(r, function(err, req, body) {
    print(body);
});

... ... ... ...
... ...
{"timestamp": 1553545586, "iss_position": {"latitude": "-22.1082", "longitude": "-92.8806"}, "message": "success"}


* Reorganizing code into functions for reuse in another cell:

In [12]:
%%node
var request = require('request');
var getPosition = function(callback) {
    var r = {
        method:'GET',
        url: 'http://api.open-notify.org/iss-now.json',
        json: true
    };
    request(r, function(err, req, body) {
        var obj = null;
        if (!err) {
            obj = body.iss_position
            obj.latitude = parseFloat(obj.latitude);
            obj.longitude = parseFloat(obj.longitude);
            obj.time = new Date().getTime();       
        }
        callback(err, obj);
    });
};

... ..... ..... ..... ..... ... ..... ..... ....... ....... ....... ....... ....... ..... ..... ...


In [13]:
%%node
getPosition(function(err, data) {
    print(data);
});

... ...
{"latitude": -21.2644, "longitude": -92.1393, "time": 1553545605014}


# Promises

* Install the NPM _request-promise_ module.
* Refactor the function a bit.
* Call it using a Promises style.

    npm.install(('request','request-promise'))

In [14]:
%%node
var request = require('request-promise');
var getPosition = function(callback) {
    var r = {
        method:'GET',
        url: 'http://api.open-notify.org/iss-now.json',
        json: true
    };
    return request(r).then(function(body) {
        var obj = null;
        obj = body.iss_position
        obj.latitude = parseFloat(obj.latitude);
        obj.longitude = parseFloat(obj.longitude);
        obj.time = new Date().getTime();         
        return obj;
    });
};

... ..... ..... ..... ..... ... ..... ..... ..... ..... ..... ..... ..... ...


In [15]:
%%node
getPosition().then(function(data) {
  print(data);
});

... ...
{"latitude": -19.1992, "longitude": -90.3768, "time": 1553545647616}


In [16]:
%%node
getPosition().then(print);

{"latitude": -18.9792, "longitude": -90.193, "time": 1553545651674}


# Sharing data between NodeJS & Python cells

Example:
Waveform time series data, stored in variable _wave_.

In [19]:
%%node
var wave = [];
for (var i = 0; i < 1000; i++) {
    var xx = 2*Math.PI * i/ 360;
    var obj = {
      xx: xx,
      i: i,
      sin: Math.sin(xx),
      cos: Math.cos(xx),
      tan: Math.tan(xx)
    };
    wave.push(obj);
}

... ... ..... ..... ..... ..... ..... ..... ... ...


    print(wave); // outputs data as JSON
    display(wave); // sends data to PixieDust visualization engine
    store(wave, w); // copies JS data to a Python variable

In [20]:
%%node
store(wave,'w');

!!! Warning: store is now deprecated - Node.js global variables are automatically propagated to Python !!!


In [21]:
print(w.count())       #count of dataframe
print(w['sin'].max())  # max value of sin wave
print(w['sin'].min())  # min value of sin wave
print(w['tan'].mean()) # avg value of tan wave

cos    1000
i      1000
sin    1000
tan    1000
xx     1000
dtype: int64
1.0
-1.0
26897141197967.586


# Accessing a Cloudant database

In [22]:
npm.install('cloudant-quickstart')

/home/bjpcjp/.nvm/versions/node/v10.6.0/bin/npm install -s cloudant-quickstart
+ cloudant-quickstart@1.25.5
updated 1 package and audited 287 packages in 3.092s
found 0 vulnerabilities


In [23]:
%%node
// create connection
var cloudantqs = require('cloudant-quickstart');
var cities = cloudantqs('https://reader.cloudant.com/cities');

In [25]:
%%node
// --------------------------------------------------------------
// explore the cities object.
// if you know document IDs, you can retrieve them individually.
// --------------------------------------------------------------

cities.get('2636749').then(print);

{"_id": "2636749", "name": "Stowmarket", "latitude": 52.18893, "longitude": 0.99774, "country": "GB", "population": 15394, "timezone": "Europe/London"}


In [26]:
%%node
// --------------------------------------------------------------
// grabbing cities data in bulk:
// --------------------------------------------------------------

cities.get(['4562407','2636749','3530597']).then(print);

[{"_id": "4562407", "name": "York", "latitude": 39.9626, "longitude": -76.72774, "country": "US", "population": 43718, "timezone": "America/New_York"}, {"_id": "2636749", "name": "Stowmarket", "latitude": 52.18893, "longitude": 0.99774, "country": "GB", "population": 15394, "timezone": "Europe/London"}, {"_id": "3530597", "name": "Mexico City", "latitude": 19.42847, "longitude": -99.12766, "country": "MX", "population": 12294193, "timezone": "America/Mexico_City"}]


In [27]:
%%node
// --------------------------------------------------------------
// example of Cloudant query function:
// --------------------------------------------------------------

cities.query({country:'GB', latitude: { "$gt": 54}}).then(display);

### Aggregating data using cloudant-quickstart

In [36]:
%%node
cities.sum('population').then(print);

2694222973


In [38]:
%%node
// return total population grouped by country:
cities.sum('population','country').then(print)

(node:7945) UnhandledPromiseRejectionWarning: #<Object>
(node:7945) UnhandledPromiseRejectionWarning: Unhandled promise rejection. This error originated either by throwing inside of an async function without a catch block, or by rejecting a promise which was not handled with .catch(). (rejection id: 1)
(node:7945) [DEP0018] DeprecationWarning: Unhandled promise rejections are deprecated. In the future, promise rejections that are not handled will terminate the Node.js process with a non-zero exit code.


In [43]:
%%node
// moving data to Python using the store function:
// NOTE: store is now deprecated, NodeJS global vars are auto-propagated to Python...
pop25kcities = cities.all({limit:25000});

In [44]:
# accessing y in a python cell
pop25kcities['population'].sum()

2694222973